# Metabolomics Data Inspection

By Garrett Roell and Christina Schenk

Tested on biodesign_3.7 kernel on jprime

This notebook gets metabolite data from the model, and uses it to attempt to match unknown metabolites in LCMS data.                                          
### Method: 
<ol>
<li>Set up imports</li>
<li>Load model and relevant data</li>
<li>Extract metabolite data from the genome scale mode</li>
<li>Check for matches between model metabolites and LCMS data</li>
</ol>


### 1. Set up imports

In [1]:
import cobra
import pandas as pd

### 2. Load model and relevant data

In [2]:
model = cobra.io.read_sbml_model("../models/r_opacus_annotated_curated.xml")

# load LCMS intracellular data (Can use other file names to get other LCMS data sets)
lcms_in_pos_df = pd.read_csv('../data/metabolomics/LCMS_intracellular_metabolites_positive.csv')
lcms_in_neg_df = pd.read_csv('../data/metabolomics/LCMS_intracellular_metabolites_negative.csv')
lcms_ex_pos_df = pd.read_csv('../data/metabolomics/LCMS_extracellular_metabolites_positive.csv')
lcms_ex_neg_df = pd.read_csv('../data/metabolomics/LCMS_extracellular_metabolites_negative.csv')

lcms_in_pos_df.head(2)

,m/z,RT [min],Name,Tags,Foston_In_1-A_Met_LC_HILIC_Pos (F4),Foston_In_1-B_Met_LC_HILIC_Pos (F5),Foston_In_1-C_Met_LC_HILIC_Pos (F6),Foston_In_1-D_Met_LC_HILIC_Pos (F7),Foston_In_1-E_Met_LC_HILIC_Pos (F8),Foston_In_1-F_Met_LC_HILIC_Pos (F9),...,Foston_In_4-C_Met_LC_HILIC_Pos (F18),Foston_In_4-D_Met_LC_HILIC_Pos (F19),Foston_In_4-E_Met_LC_HILIC_Pos (F20),Foston_In_4-F_Met_LC_HILIC_Pos (F21),Foston_In_4-G_Met_LC_HILIC_Pos (F22),Foston_In_4-H_Met_LC_HILIC_Pos (F23),Foston_In_4-I_Met_LC_HILIC_Pos (F24),Foston_In_4-J_Met_LC_HILIC_Pos (F25),Foston_In_4-K_Met_LC_HILIC_Pos (F26),Foston_In_4-L_Met_LC_HILIC_Pos (F27)
0,102.05471,5.01,1-Aminocyclopropane-1-carboxylic acid,"Confirmed ID (HIgh Confidence); Match Mass, MS...",8.797964e+07,8.488728e+07,8.768286e+07,8.051226e+07,7.362946e+07,7.990522e+07,...,7.814846e+07,7.662056e+07,8.191530e+07,8.115013e+07,7.647841e+07,6.723747e+07,5.797746e+07,5.682586e+07,6.747358e+07,5.969872e+07
1,150.07725,3.07,1-Methyladenine,"Confirmed ID (HIgh Confidence); Match Mass, MS...",6.416126e+05,5.023436e+05,5.055905e+05,6.324821e+05,9.560859e+05,4.266593e+05,...,4.152413e+06,9.243055e+06,8.288173e+06,8.352531e+06,4.783644e+06,4.330093e+06,3.890677e+06,3.866626e+06,4.901932e+06,4.933419e+06


### 3a. Extract metabolite data from the genome scale model
This was done outside of jprime becuase lxml library was needed and is not in the Biodesign kernel

In [3]:
# # helper function for getting molecular weight from metanetx.org
# def get_metanetx_molecular_weight(metanetx_id):
#     url = f'https://www.metanetx.org/chem_info/{metanetx_id}'
    
#     metanetx_df = pd.read_html(url)[1]
    
#     mass_row = metanetx_df[metanetx_df["Unnamed: 0"] == 'mass']
    
#     molecular_weight = float(mass_row.Properties.values[0])
    
#     return molecular_weight

# get_metanetx_molecular_weight('MNXM61')

In [4]:
# # create a list to hold metabolite data
# row_data = []

# # loop over the metabolites in the model
# for m in model.metabolites:
#     print(m)
    
#     # get MetaNetX id if present
#     if 'metanetx.chemical' in m.annotation.keys():
#         metanetx_id = m.annotation['metanetx.chemical']
#         metanetx_molecular_weight = get_metanetx_molecular_weight(metanetx_id)
#     else:
#         metanetx_id = ''
#         metanetx_molecular_weight = ''
    
#     # get KEGG id if present
#     if 'kegg.compound' in m.annotation.keys():
#         kegg_id = m.annotation['kegg.compound']
#     else:
#         kegg_id = ''
        
#     # create a dictionary for each metabolite's information
#     row_data.append({
#         "metanetx_molecular_weight": metanetx_molecular_weight,
#         "formula_molecular_weight": m.formula_weight,
#         "name": m.name,
#         "formula": m.formula,
#         "metabolite_id": m.id,
#         "metanetx_id": metanetx_id,
#         "kegg_id": kegg_id,
#     })
    
# # convert the row data into a data frame
# metabolite_df = pd.DataFrame(row_data)

# # sort by molecular weight
# metabolite_df.sort_values(by=['formula_molecular_weight'], inplace=True)

# metabolite_df.head(5)

Save metabolite data from the model as a csv

In [5]:
# metabolite_df.to_csv('../data/metabolomics/model_metabolites.csv', index=False, header=True)

### 3b. Load metabolite data from csv

In [6]:
metabolite_df = pd.read_csv('../data/metabolomics/model_metabolites.csv')
metabolite_df.head(10)

,metanetx_molecular_weight,formula_molecular_weight,name,formula,metabolite_id,metanetx_id,kegg_id
0,NaN,0.00000,Plastoquinol,NaN,pqh2_p,NaN,NaN
1,NaN,0.00000,Ferrocytochrome c6,NaN,focytc6_p,NaN,NaN
2,NaN,0.00000,Ferricytochrome c6,NaN,ficytc6_p,NaN,NaN
3,NaN,0.00000,Plastoquinone,NaN,pq_p,NaN,NaN
4,1.00794,1.00794,H+,H,h_c,MNXM1,C00080
5,1.00794,1.00794,H+,H,h_e,MNXM1,C00080
6,1.00794,1.00794,H+,H,h_p,MNXM1,C00080
7,2.01565,2.01588,Hydrogen,H2,h2_c,MNXM195,C00282
8,NaN,15.01464,N 2 methyl 3 5 dinitrophenyl 4 methyl 3 5 di...,HN,2m35mdntha_c,NaN,NaN
9,NaN,15.01464,N N bis 3 5 dinitrotolyl amine,HN,35dnta_c,NaN,NaN


### 4. Check for matches between model metabolites and LCMS data

In [7]:
hydrogen_mass = 1.008

# define a helper function to get the model metabilte data from a given molecular weight
def molecular_weight_to_metabolite_data(molecular_weight):

    # keep track of the closest mass distance between the given 
    # molecular weight and model metabolite's molecular weight
    minimum_mass_difference = 1000
    
    # define an arbitrary closest metabolite
    closest_molecular_weight_data = metabolite_df.loc[0]
    
    # loop over metabolite data
    for _, row in metabolite_df.iterrows():
        
        # get the metabolite mass values for with or without a proton
        metabolite_molecular_weight = row.metanetx_molecular_weight
        metabolite_plus_1_molecular_weight = row.metanetx_molecular_weight + hydrogen_mass
        metabolite_minus_1_molecular_weight = row.metanetx_molecular_weight + hydrogen_mass
        
        # check if this metabolite is the closest in mass to the given molecular weight
        if abs(metabolite_molecular_weight - molecular_weight) < minimum_mass_difference:
            # if so, the update the data for the the closest metabolite and the min mass distance
            closest_molecular_weight_data = row
            minimum_mass_difference = abs(metabolite_molecular_weight - molecular_weight)
            
        # check this metabolite plus a proton
        elif abs(metabolite_plus_1_molecular_weight - molecular_weight) < minimum_mass_difference:
            closest_molecular_weight_data = row
            minimum_mass_difference = abs(metabolite_plus_1_molecular_weight - molecular_weight)
            
        # check this metabolite minus a proton
        elif abs(metabolite_minus_1_molecular_weight - molecular_weight) < minimum_mass_difference:
            closest_molecular_weight_data = row
            minimum_mass_difference = abs(metabolite_minus_1_molecular_weight - molecular_weight)

    # return the data from the metabolite with the closest molecular weight
    return closest_molecular_weight_data, minimum_mass_difference

# a testing function
molecular_weight_to_metabolite_data(148.06024)

(metanetx_molecular_weight              147.053
 formula_molecular_weight               147.129
 name                         O-Acetyl-L-serine
 formula                                C5H9NO4
 metabolite_id                          acser_c
 metanetx_id                            MNXM418
 kegg_id                                 C00979
 Name: 528, dtype: object, 0.0009200000000362252)

In [8]:
# A helper function to get lists of model metabolites and mass differences for a given lcms dataframe
def get_model_metabolites_and_mass_differences(lcms_df):
    
    # create lists to store values
    model_metabolites = []
    mass_differences = []

    # loop over metabolites that have LCMS measurements
    for _, row in lcms_df.iterrows():    
        # get data from 
        molecular_weight = row['m/z']
        model_metabolite, mass_difference = molecular_weight_to_metabolite_data(molecular_weight)

        model_metabolites.append(model_metabolite['name'])
        mass_differences.append(mass_difference)
        
    return model_metabolites, mass_differences

Add columns for predicted model metabolites and mass differences to the intracellular positive LCMS data

In [9]:
model_metabolites, mass_differences = get_model_metabolites_and_mass_differences(lcms_in_pos_df)
lcms_in_pos_df.insert(3, "model_metabolite", model_metabolites)
lcms_in_pos_df.insert(4, "mass_difference", mass_differences)

Add columns for predicted model metabolites and mass differences to the intracellular negative LCMS data

In [10]:
model_metabolites, mass_differences = get_model_metabolites_and_mass_differences(lcms_in_neg_df)
lcms_in_neg_df.insert(3, "model_metabolite", model_metabolites)
lcms_in_neg_df.insert(4, "mass_difference", mass_differences)

Add columns for predicted model metabolites and mass differences to the extracellular positive LCMS data

In [11]:
model_metabolites, mass_differences = get_model_metabolites_and_mass_differences(lcms_ex_pos_df)
lcms_ex_pos_df.insert(3, "model_metabolite", model_metabolites)
lcms_ex_pos_df.insert(4, "mass_difference", mass_differences)

Add columns for predicted model metabolites and mass differences to the extracellular negative LCMS data

In [12]:
model_metabolites, mass_differences = get_model_metabolites_and_mass_differences(lcms_ex_neg_df)
lcms_ex_neg_df.insert(3, "model_metabolite", model_metabolites)
lcms_ex_neg_df.insert(4, "mass_difference", mass_differences)

Save updated lcms data as dataframes

In [13]:
lcms_in_pos_df.to_csv('../data/metabolomics/LCMS_intracellular_metabolites_positive_with_model.csv', index=False, header=True)
lcms_in_neg_df.to_csv('../data/metabolomics/LCMS_intracellular_metabolites_negative_with_model.csv', index=False, header=True)
lcms_ex_pos_df.to_csv('../data/metabolomics/LCMS_extracellular_metabolites_positive_with_model.csv', index=False, header=True)
lcms_ex_neg_df.to_csv('../data/metabolomics/LCMS_extracellular_metabolites_negative_with_model.csv', index=False, header=True)